In [ ]:
pwd

In [1]:
import os
from Utils.read_coord import kml_coord
from Utils.sentinal_request import senti_api
from Utils.forest_cover import get_cover, green_cover_plot
from Utils.seques_forecast import plot_SEQ,cal_existing
from Utils.map_view import map_show
from Utils.tree_count import count_trees
from Utils.tree_count import tree_count

import numpy as np
import pandas as pd


def count_trees(res,green_pixel,spacing):
    
    # Count no of trees per pixel
    tree_per_pixel = (res*res)/(spacing*spacing)
    tree_count = round(green_pixel*tree_per_pixel)
    
    print('Estimated No of Trees: ',tree_count)
    return tree_count

In [ ]:
spacing = 4
count_trees(resol,green_pix,spacing)

## Check a pipeline

In [2]:
file_path = input('Enter kml file path: ')
print(file_path)
#C:\Users\DELL\Documents\GitHub\SPE\SPE-testing\Test Data\gAMPOLA TEST SITE 2.kml
if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()

        #print(lines)
else:
    print('The specified file does NOT exist')
    
# getting polygon coordinates
poly_coord = kml_coord(file_path)
#print(poly_coord)

# show the map view
map_show(poly_coord)

    
# User inputs
print("Forecasting for one tree type")
tree_type = input('Enter the tree type: ')
n_tree = input('Enter the tree count: ')
n_tree = int(n_tree)



img,resol = senti_api(poly_coord)
print('resolution',resol)
#print(img.shape)
#print(img[:,:,2][40][10])
forest_area,forest_cover,green_pix,green_cov,ndvi = get_cover(img,resol)
print('green_pix',green_pix)
print('green_cover',green_cov)
print('forest_cover',forest_cover)

# CO2 forecasting
print(tree_type)
plot_SEQ(tree_type,n_tree)

spacing = input('Enter the spacing between the trees in meters: ')
spacing = int(spacing)


if spacing == 0:
    age_tree = input('Enter the average age of trees: ')
    age_tree = int(age_tree)

    t_count = tree_count(tree_type, green_cov, age_tree)
else:
    t_count = count_trees(resol,green_pix,spacing)
    
print("Estimated Number of Trees",t_count)

fig = green_cover_plot(ndvi,forest_cover,t_count)

Enter kml file path: C:\Users\Harith\EngenuityAI\SPE\SPE testing\SPE-testing\Test Data\gAMPOLA TEST SITE 2.kml
C:\Users\Harith\EngenuityAI\SPE\SPE testing\SPE-testing\Test Data\gAMPOLA TEST SITE 2.kml
The file exists
Forecasting for one tree type
Enter the tree type: Rubber
Enter the tree count: 30
resolution 10
green_pix 2522
green_cover 0.2522
forest_cover 89.02
Rubber


TypeError: plot_SEQ() missing 1 required positional argument: 'age_max'

In [13]:
age_max = input('Enter the no of years ahead to forecast: ')
age_max = int(age_max)


Enter the no of years ahead to forecast: 25


In [14]:
t_count=30

In [15]:
seq = cal_existing(age_max,t_count,tree_type)

In [16]:
seq

Series([], dtype: float64)

In [17]:
print(seq.item())

ValueError: can only convert an array of size 1 to a Python scalar

In [8]:
x

4.122544898469472

In [12]:
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cv2
import rasterio

import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from matplotlib.colors import ListedColormap
import imutils

def green_cover_plot(ndvi,green_cover,thresh = 0.6):
    
    '''
    ndvi: ndvi index values

    '''
    
    # Create classes and apply to NDVI results
    ndvi_class_bins = [-np.inf, thresh, np.inf]
    ndvi_landsat_class = np.digitize(ndvi, ndvi_class_bins)

    # Apply the nodata mask to the newly classified NDVI data
    ndvi_landsat_class = np.ma.masked_where(np.ma.getmask(ndvi), 
                                            ndvi_landsat_class)
    #np.unique(ndvi_landsat_class)
    
    # Define color map
    nbr_colors = ["gray", "g"]
    nbr_cmap = ListedColormap(nbr_colors)

    # Define class names
    ndvi_cat_names = [
        "No Vegetation",
        "Vegetation"]

    # Get list of classes
    classes = np.unique(ndvi_landsat_class)
    classes = classes.tolist()
    
    # The mask returns a value of none in the classes. remove that
    classes = classes[0:2]

    # Plot your data
    fig, ax = plt.subplots()
    im = ax.imshow(ndvi_landsat_class, cmap=nbr_cmap)
    ep.draw_legend(im_ax=im, classes=classes, titles=ndvi_cat_names)
    ax.set_title(
    "Forest Cover - "+ str(green_cover)+"%",
    fontsize=14)
    ax.set_axis_off()
    plt.show()
    #canvas = plt.gca().figure.canvas
    #canvas.draw()
    #data = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
    #image = data.reshape(canvas.get_width_height()[::-1] + (3,))
    
    #save img
    #veg_file = os.path.splitext(file_name)[0]+'_g'+'.png'
    #plt.savefig(save_path+'/'+veg_file)
    #print(veg_file+'\n'+"successfully saved!")

    return fig

In [ ]:
 Define color map
nbr_colors = ["tan", "forestgreen"]#, "burlywood"]
nbr_cmap = ListedColormap(nbr_colors)

# Define class names
ndvi_cat_names = [
    "No Vegetation",
    "Vegetation",
]

# Get list of classes
classes = np.unique(ndvi_landsat_class)
classes = classes.tolist()
# The mask returns a value of none in the classes. remove that
classes = classes[0:2]

In [ ]:
# Plot your data
fig, ax = plt.subplots()
im = ax.imshow(ndvi_landsat_class, cmap=nbr_cmap)
textstr = '\n'.join((
    r'Forest cover: %.2f' % (green_cov, ),
    r'Total Sequestared CO2 amount: %.2f' % (seq, ),
    r'Estimated No of trees: %.2f$' % (t_count, )))

ep.draw_legend(im_ax=im, classes=classes, titles=ndvi_cat_names)
ax.set_title(
    "Forest Cover - "+ str(green_cover)+"%",
    fontsize=14,
)
ax.set_axis_off()
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax.text(1.05, 0.80, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)

# Auto adjust subplot to fit figure size
plt.tight_layout()